# Sprint
## 公開されている実装を動かす

## 1.このSprintについて

### Sprintの目的
- 公開されている実装を動かす際のノウハウを得る
- 論文で読んだ手法を動かしてみる

### どのように学ぶか
論文で読んだ手法の公開されている実装を実際に動かしていきます。

In [1]:
# tensorflow準備
!pip install tensorflow==1.15.0
import tensorflow
print(tensorflow.__version__)
# driveマウント
from google.colab import drive
drive.mount('/content/drive/')

1.15.0
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# kaggle.json準備
!pip install kaggle
!mkdir -p /content/.kaggle
!\cp -f '/content/drive/My Drive/DIVEINTOCODE/kaggle.json' /content/.kaggle/kaggle.json
!chmod 600 /content/.kaggle/kaggle.json
!mkdir -p /root/.kaggle
!\cp -Rf /content/.kaggle/kaggle.json /root/.kaggle/
!ls -al /content/.kaggle/
!ls -al /root/.kaggle/

total 12
drwxr-xr-x 2 root root 4096 Jun 22 06:26 .
drwxr-xr-x 1 root root 4096 Jun 22 06:26 ..
-rw------- 1 root root   67 Jun 22 08:42 kaggle.json
total 16
drwxr-xr-x 2 root root 4096 Jun 22 06:26 .
drwx------ 1 root root 4096 Jun 22 06:45 ..
-rw------- 1 root root   67 Jun 22 08:42 kaggle.json


## 2.Faster R-CNN

Faster R-CNN\[1\]の実装を動かします。


\[1\]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99


[https://arxiv.org/pdf/1506.01497.pdf](https://arxiv.org/pdf/1506.01497.pdf)


以下のものを使用してください。Kerasを使用した実装です。


[duckrabbits/ObjectDetection at master](https://github.com/duckrabbits/ObjectDetection/tree/master)

In [6]:
import json
import os

### 【問題1】
#### 学習と推定
READMEを参考に上記実装を動かしてください。

In [5]:
# カレントディレクトリの変更
os.chdir('/content/drive/My Drive/')
# カレントディレクトリの取得
print(os.getcwd())
!rm -rf ObjectDetection
!git clone https://github.com/duckrabbits/ObjectDetection.git
# データセット準備
!rm -rf Simpsons
!kaggle datasets download -d alexattia/the-simpsons-characters-dataset --force
!unzip /content/drive/'My Drive'/the-simpsons-characters-dataset.zip  -d Simpsons
!mv Simpsons/simpsons_dataset ObjectDetection/

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1271.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1272.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1273.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1274.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1275.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1276.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1277.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1278.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1279.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1280.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset/moe_szyslak/pic_1281.jpg  
  inflating: Simpsons/simpsons_dataset/simpsons_dataset

In [10]:
# 学習
os.chdir('/content/drive/My Drive/ObjectDetection/')
print(os.getcwd())
!python train.py -p annotation.txt --n_epochs 100 --n_iters 10

/content/drive/My Drive/ObjectDetection
Using TensorFlow backend.
Parsing annotation files
Training images per class (19 classes) :
{'abraham_grampa_simpson': 687,
 'apu_nahasapeemapetilon': 206,
 'bart_simpson': 650,
 'bg': 0,
 'charles_montgomery_burns': 650,
 'chief_wiggum': 209,
 'comic_book_guy': 208,
 'edna_krabappel': 212,
 'homer_simpson': 718,
 'kent_brockman': 213,
 'krusty_the_clown': 429,
 'lisa_simpson': 756,
 'marge_simpson': 629,
 'milhouse_van_houten': 210,
 'moe_szyslak': 403,
 'ned_flanders': 675,
 'nelson_muntz': 219,
 'principal_skinner': 614,
 'sideshow_bob': 201}
-------------------------------
path to config file : ./save/train_20200622-091543_config.pickle
-------------------------------
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to  

In [ ]:
!rm -rf output

In [ ]:
!python predict.py -i simpsons_dataset/troy_mcclure/ -c save/train_20200622-064438_config.pickle

### 【問題2】
#### コードリーディング
上記実装のコードリーディングを行ってください。


まず、Faster R-CNN\[1\]において重要だと考えた部分を列挙してください。そして、それに対応するコードを見つけてください。


（例）


- RPNを実現しているコードはどこか
- RoIプーリングを実現しているコードはどこか

フレームワークには畳み込み層など一般的なものはクラスが用意されていますが、RoIプーリングなど特定の手法限定のものは用意されていません。オリジナルのレイヤーを作成することが可能であり、Kerasであれば以下のページに情報がまとまっています。


[オリジナルのKerasレイヤーを作成する - Keras Documentation](https://keras.io/ja/layers/writing-your-own-keras-layers/)


**《参考》**


KerasではVGG16のクラスが用意されているため、簡単に利用ができます。`include_top=False`の引数を与えることで、出力のための全結合層部分が除かれます。`weights='imagenet'`でImageNetを利用した学習済みモデルも手に入り、転移学習 が行えます。`weights='None'`とすればランダムな初期化となります。


[Applications - Keras Documentation](https://keras.io/ja/applications/#vgg16)



- RPN実装
- AnchorBoxのパラメータ
- RoIプーリング

### 3.YOLOv3

シンプソンズのデータセットをFaster R-CNN以外の手法で学習・推定を行います。YOLOv3\[2\]のKeras実装を使います。


[qqwweee/keras-yolo3: A Keras implementation of YOLOv3 (Tensorflow backend)](https://github.com/qqwweee/keras-yolo3)


\[2\]Jeseph Redmon, Ali Farhadi. YOLOv3: An Incremental Improvement


[https://pjreddie.com/media/files/papers/YOLOv3.pdf](https://pjreddie.com/media/files/papers/YOLOv3.pdf)

### 【問題3】
#### 学習済みの重みによる推定
学習済みの重みを使い推定を行う方法がREADME.mdのQuick Startの項に記載されています。


まずはこの通りにして各自何かしらの画像や動画に対して検出を行ってください。


出力結果を課題の一部として提出してください。

In [ ]:
os.chdir('/content/drive/My Drive/')
print(os.getcwd())

### 【問題4】
#### 学習のためのファイルを作成
新しいデータ（シンプソンズデータセット）を学習します。README.mdのTrainingの項を読み、シンプソンズデータセットを学習するために必要なファイルを作成してください。


アノテーションファイルの形式が問題1の実装とは異なるため変換する必要があります。

### 【問題5】
#### 学習が行えることの確認
問題4で作成したファイルを使用して学習してください。実行環境で学習に時間がかかる場合は、学習が行えることを確認するのみで終えて構いません。

### 【問題6】
#### （アドバンス課題）コードリーディング
上記実装のコードリーディングを行ってください。


まず、YOLOv3\[2\]の論文において重要だと考えた部分を列挙してください。そして、それに対応するコードを見つけてください。